In [1]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

## Prepare data

In [3]:
har_x_train, har_y_train, list_ch_train = read_data(data_path="./dataset/UCI HAR Dataset/", split="train") # train
har_x_test, har_y_test, list_ch_test = read_data(data_path="./dataset/UCI HAR Dataset/", split="train") # test

In [4]:
har_x_train, har_x_test = standardize(har_x_train, har_x_test)

In [5]:
#har_y_train = har_y_train - 1

In [6]:
#har_y_test = har_y_test - 1

In [7]:
X_train, X_valid, y_train, y_valid = train_test_split(har_x_train, har_y_train, stratify=har_y_train, random_state=42)

In [8]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

# Build CNN model

## Hyperparameters

In [230]:
batch_size = 60
seq_len = 128
learning_rate = 0.01
n_epochs = 100

n_classes = 7 # from 1 - 6
n_channels = 9

## the model

In [231]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")

# learning rate exponential decay
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.01
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100, 0.96, staircase=True)

In [232]:
with tf.name_scope("cnn"):
    
    # Convolutional layers
    conv1 = tf.layers.conv1d(X, 
                             filters=32, 
                             kernel_size=2, 
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)

    conv2 = tf.layers.conv1d(conv1,
                             filters=64, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)

    conv3 = tf.layers.conv1d(conv1,
                             filters=96, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    
    last_shape = conv3.get_shape().as_list()
    print(last_shape)
    # Fully Connected layers
    flat = tf.reshape(conv3, (-1, last_shape[1] * last_shape[2]))
    #drop = tf.nn.dropout(flat, keep_prob=1.0, training=training)
    
    fc1 = tf.layers.dense(flat, 100)
    bn_fc1 = tf.layers.batch_normalization(fc1, training=training)
    bn_fc1_act = tf.nn.relu(bn_fc1)
    
    fc2 = tf.layers.dense(bn_fc1_act, 100)
    bn_fc2 = tf.layers.batch_normalization(fc2, training=training)
    bn_fc2_act = tf.nn.relu(bn_fc2)
    
    logits_before_bn = tf.layers.dense(bn_fc2_act, n_classes)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training)
    #logits = tf.layers.dense(bn_fc2_act, n_classes)
    
    

[None, 128, 96]


In [233]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(7)])

In [234]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [235]:
with tf.name_scope("train"):
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss, global_step=global_step)

In [236]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [237]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [238]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for X_batch, y_batch in get_batches(X_train, y_train, batch_size):
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch,"Learning rate:", sess.run(optimizer._lr), "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_model.ckpt")

Epoch:  0 Learning rate: 0.01 Valid accuracy:  0.156692
Epoch:  10 Learning rate: 0.00664832 Valid accuracy:  0.873232
Epoch:  20 Learning rate: 0.00460419 Valid accuracy:  0.875408
Epoch:  30 Learning rate: 0.00318856 Valid accuracy:  0.87704
Epoch:  40 Learning rate: 0.00220818 Valid accuracy:  0.876496
Epoch:  50 Learning rate: 0.00152924 Valid accuracy:  0.877584
Epoch:  60 Learning rate: 0.00105905 Valid accuracy:  0.877584
Epoch:  70 Learning rate: 0.000733429 Valid accuracy:  0.879761
Epoch:  80 Learning rate: 0.000507925 Valid accuracy:  0.879761
Epoch:  90 Learning rate: 0.000351755 Valid accuracy:  0.879761


In [239]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model.ckpt")
    test_acc = accuracy.eval(feed_dict={X: har_x_test, y: har_y_test})
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model.ckpt
Test Accuracy:  0.968852
